In [1]:
import pandas as pd
df = pd.read_csv('cluster_forumPostNames.csv')
df.describe()

,Category,Topic
count,352,352
unique,5,294
top,Relationships and Love,Замужество - это обслуживание мужика?
freq,138,4


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
russian_stop_words = stopwords.words('russian')

texts = df['Topic'].tolist()

#TF-IDF
vectorizer = TfidfVectorizer(stop_words=russian_stop_words)
X = vectorizer.fit_transform(texts)

num_clusters = 5

kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(X)

numeric_labels = kmeans.labels_

label_mapping = {0: 'Career and Finance', 1: 'Health and Wellness', 2: 'Parenting and Family', 3: 'Relationships and Love', 4: 'Societal Issues and Morality'}

desired_labels = [label_mapping[label] for label in numeric_labels]

centroids = kmeans.cluster_centers_


terms = vectorizer.get_feature_names_out()
for i, centroid in enumerate(centroids):
    top_terms_idx = centroid.argsort()[-20:][::-1]
    top_terms = [terms[idx] for idx in top_terms_idx]
    print(f"{label_mapping[i]} Cluster: {top_terms}")


Career and Finance Cluster: ['мужчиной', 'стали', 'прозревшим', 'вступать', 'отношения', '50', 'расстаться', 'женатым', 'помогите', '40тыс', 'заводить', 'зарплате', '35', 'принять', 'расходы', 'иностранцем', 'сложно', 'ребенка', 'девушки', 'мужа']
Health and Wellness Cluster: ['мужчина', 'полов', 'муж', 'секс', 'жизнь', 'работа', 'отзывы', 'болезнь', 'это', 'против', 'равноправие', 'ринопластика', 'найти', 'лет', 'стюардессой', 'справляетесь', 'паркинсона', 'хочет', 'лечение', 'обслуживание']
Parenting and Family Cluster: ['почему', 'детей', 'россии', 'женщины', 'рожать', 'хотеть', 'перестали', 'хочу', 'пары', 'финансово', 'женятся', 'молодые', 'рожают', 'несостоявшиеся', 'жить', 'плюсы', 'лет', 'форумчане', 'живёте', 'замужем']
Relationships and Love Cluster: ['мужа', 'встречаете', 'работы', 'жена', 'ребенок', 'своего', 'укротить', 'партнеров', 'любовницей', 'любовницу', 'ищу', 'бороться', 'внебрачный', 'кастрировать', 'узнала', 'любовнице', 'моего', 'дочь', 'ненавидит', 'сколько']
So

In [18]:
#В этом коде x_train и x_test получаются с использованием train_test_split, который разбивает исходные данные X на обучающий и тестовый наборы. Затем кластеризация (kmeans.predict) выполняется отдельно для обучающего и тестового наборов, и метки кластеров сохраняются в y_train и y_test соответственно через all_predictions - это массив с предсказанными метками кластеров для всех данных X
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

nltk.download('stopwords', quiet=True)
russian_stop_words = stopwords.words('russian')

def cluster_and_evaluate(X, num_clusters=5, test_size=0.4, random_state=42):

    kmeans = KMeans(n_clusters=num_clusters, random_state=random_state, n_init=10)
    kmeans.fit(X)

    all_predictions = kmeans.predict(X)

    x_train, x_test, y_train, y_test = train_test_split(X, all_predictions, test_size=test_size, random_state=random_state)

    train_predictions = kmeans.predict(x_train)

    test_predictions = kmeans.predict(x_test)

    train_accuracy = accuracy_score(y_train, train_predictions)
    train_precision = precision_score(y_train, train_predictions, average='weighted')
    train_recall = recall_score(y_train, train_predictions, average='weighted')

    test_accuracy = accuracy_score(y_test, test_predictions)
    test_precision = precision_score(y_test, test_predictions, average='weighted')
    test_recall = recall_score(y_test, test_predictions, average='weighted')

    return train_accuracy, train_precision, train_recall, test_accuracy, test_precision, test_recall

texts = df['Topic'].tolist()

vectorizer = TfidfVectorizer(stop_words=russian_stop_words)
X = vectorizer.fit_transform(texts)

num_clusters = 5

train_accuracy, train_precision, train_recall, test_accuracy, test_precision, test_recall = cluster_and_evaluate(X, num_clusters)

print("Train Accuracy:", train_accuracy)
print("Train Precision:", train_precision)
print("Train Recall:", train_recall)

print("\nTest Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)


Train Accuracy: 1.0
Train Precision: 1.0
Train Recall: 1.0

Test Accuracy: 1.0
Test Precision: 1.0
Test Recall: 1.0
